code by Jakob

In [ ]:
#! pip install requests

In [ ]:
import pandas as pd
import requests 
import xml.etree.ElementTree as ET
import time
import logging

In [2]:
data_dir = "data/"
dataframe_dir = data_dir + "dataframes/"
speeches_dir = data_dir + "speeches/"
speeches_pdf_dir = speeches_dir + "pdf/"
last_metadata_step = 'metadata_s01'
current_metadata_step = 'metadata_s02'
metadata_intermediate_step = 'metadata_s02_interm'

In [157]:
base_url = "https://digitallibrary.un.org/search"
namespace = {'nmsp': 'http://www.loc.gov/MARC21/slim'}
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
# logging.basicConfig(filename='logs/example.log', encoding='utf-8', level=logging.INFO)
crawl_delay = 5 # from robots.txt
log.info(f"log with crawl delay {crawl_delay}s")

INFO:__main__:log with crawl delay 5s


In [ ]:
filter_only_doc = "fct__1=Documents+and+Publications"
filter_marcxml = "of=xm"
result_max = "rg=100"
search_url = lambda param: f"{base_url}?ln=en&p={param}&f=&rm=&sf=&so=d&rg=50&c=United+Nations+Digital+Library+System&{filter_marcxml}&fti=0&{filter_only_doc}&fti=0"

# minimal search would be: 
search_url = lambda param: f"{base_url}?p={param}&ln=en&{filter_marcxml}&{filter_only_doc}&{result_max}"

# Call search with document symbol to obtain document url

In [ ]:
def search_doc_for_document_symbol(data, document_symbol, failed_requests, unmatched_requests, crawl_delay=crawl_delay, session=None, match_starts_same=False): 
    try:
        # pause between requests
        time.sleep(crawl_delay)
        record_id, document_symbol_found = None, None
        languages = []
        record_found = False
        current_results = len(data)

        url = search_url(document_symbol)
        # They introduced AWS challenges throughout the duration of our project so we have to use session-less requests.
        if session:
            search_res = session.get(url)
        else:
            search_res = requests.get(url)

        if search_res.status_code == 202: # we experienced some 202 issues while testing - although they seme to be gone now
            log.warning(f"Web page returned status code: {search_res.status_code}, will wait shorly and try again. url was {url}")
            time.sleep(crawl_delay)
            if session:
                search_res = session.get(url)
            else:
                search_res = requests.get(url)
        if search_res.status_code != 200:
            log.error(f"Web page returned status code: {search_res.status_code} for search of {document_symbol} on url {url}")
            failed_requests.append(document_symbol)
            return

        root = ET.fromstring(search_res.text)

        # extract needed fields from the marcxml
        # main objective is the english pdf link
        for record in root.findall('nmsp:record', namespace):
            record_id = record.find("nmsp:controlfield[@tag='001']", namespace)
            field = record.find("nmsp:datafield[@tag='191']", namespace)
            document_symbol_found = field.find("nmsp:subfield[@code='a']", namespace)

            dsl, fsl = document_symbol.lower(), document_symbol_found.text.lower()
            exact_match = document_symbol_found.text.lower() == dsl
            starts_same = fsl.startswith((f"{dsl} ", f"{dsl}(", f"{dsl}-"))
            if (exact_match or (match_starts_same and starts_same)) and not record_found:
                field = record.find("nmsp:datafield[@tag='992']", namespace)
                if field is None:
                    field = record.find("nmsp:datafield[@tag='269']", namespace) # only year
                    logging.warning(f"The record {document_symbol_found.text} will only saved with date as year")
                date = field.find("nmsp:subfield[@code='a']", namespace)
            
                downloadfields = record.findall("nmsp:datafield[@tag='856']", namespace)
                if len(downloadfields) == 0:
                    log.warning(f"No download links for document_symbol: {document_symbol} where found {document_symbol_found.text} and id: {record_id.text}")
                    continue
                else:
                    for downloadfield in downloadfields:
                        lang_field = downloadfield.find("nmsp:subfield[@code='y']", namespace)
                        if lang_field is None:
                            log.info(f"lang field unfilled for datafield {downloadfield.text}")
                            continue
                        elif lang_field.text.lower() == 'english':
                            document_url = downloadfield.find("nmsp:subfield[@code='u']", namespace)
                            data.append([
                                record_id.text, 
                                document_symbol_found.text, 
                                lang_field.text,
                                document_url.text,
                                date.text])
                            record_found = True
                            break
                        else:
                            languages.append(lang_field.text)
                            # mainly non matching languages - can be log.debug but not needed
                            # log.warning(f"Encountered some lang_field problem for id: {record_id.text} document_symbol: {document_symbol}")
                            continue
                
                log.info(f"\tMatched {document_symbol} to the found: {document_symbol_found.text} and record_id {record_id.text}")
            else:
                log.debug(f"\tcould not match {document_symbol} to the found: {document_symbol_found.text}")
                continue
    except Exception as e:
        log.error(f"while searching {document_symbol}")
        log.error(f"called url was {url}")
        # log.info(f" when last id: {record_id}")
        log.info(f" when last id: {record_id.text if (record_id is not None) else 'unknown'}")
        log.info(f" last document symbol: {document_symbol_found.text if (document_symbol_found is not None) else 'unknown'}")
        log.exception(e)
        failed_requests.append(document_symbol)
    
    if (len(data)-current_results) == 0:
        if len(languages) == 0:
            log.warning(f"No results found for {document_symbol} on url {url}")
        else:
            log.warning(f"No matching languages found for {document_symbol}, existing lang {languages} on url {url}")
        unmatched_requests.append(document_symbol)


## 1. search-crawl iteration to obtain urls from exact match

In [ ]:
data = []
failed_requests, unmatched_requests = [],[]
speech_records = pd.read_csv(f'{dataframe_dir}{last_metadata_step}.csv')
# session = create_session() # They introduced AWS challenges throughout the duration of our project so we have to use session-less requests.
log.info(f'Has {len(speech_records)} records to search the url of')

for speech_record in speech_records.itertuples():
    search_doc_for_document_symbol(data, speech_record.document_symbol, failed_requests, unmatched_requests)

if len(failed_requests)>0:
    log.warning(f"The following searches failed: {failed_requests}")
if len(unmatched_requests)>0:
    log.warning(f"The following searches found no matching results: {unmatched_requests}")

INFO:__main__:Has 1205 records to search the url of
INFO:__main__:	Matched S/PV.9988 to the found: S/PV.9988 and record_id 4089910
INFO:__main__:	Matched S/PV.9962 to the found: S/PV.9962 and record_id 4086244
INFO:__main__:	Matched S/PV.9941 to the found: S/PV.9941 and record_id 4085074
INFO:__main__:	Matched S/PV.9939 to the found: S/PV.9939 and record_id 4084933
INFO:__main__:	Matched S/PV.9938 to the found: S/PV.9938 and record_id 4084657
INFO:__main__:	Matched A/79/PV.72 to the found: A/79/PV.72 and record_id 4084153
INFO:__main__:	Matched S/PV.9919 to the found: S/PV.9919 and record_id 4083197
INFO:__main__:	Matched A/79/PV.67 to the found: A/79/PV.67 and record_id 4082015
INFO:__main__:	Matched S/PV.9907 to the found: S/PV.9907 and record_id 4081571
INFO:__main__:	Matched S/PV.9884 to the found: S/PV.9884 and record_id 4079464
INFO:__main__:	Matched S/PV.9861 to the found: S/PV.9861 and record_id 4077112
INFO:__main__:	Matched S/PV.9841 to the found: S/PV.9841 and record_id 4073

info about previous output: in this previous code run (2h) the failed searches correspond to not having the english language and then the code throwing and error. The code is now updated and the error is prevented to happen again.

In [ ]:
len(data)

1177

### 2. search crawl iteration - with match start active and retry failures
and one that still had a wrong name

In [ ]:
speech_records_2 = speech_records[speech_records.document_symbol.isin(list(unmatched_requests))].copy()
speech_records_2.loc[speech_records_2.document_symbol == 'S/PV.2399andCorr.1', 'document_symbol'] = 'S/PV.2399'

In [ ]:
data_2 = []
failed_requests_2, unmatched_requests_2 = [],[]
log.info(f'Has {len(speech_records_2)} records to search the url of')

for speech_record in speech_records_2.itertuples():
    search_doc_for_document_symbol(data_2, speech_record.document_symbol, failed_requests_2, unmatched_requests_2, match_starts_same=True)

if len(failed_requests_2)>0:
    log.warning(f"The following searches failed: {failed_requests_2}")
if len(unmatched_requests_2)>0:
    log.warning(f"The following searches found no matching results: {unmatched_requests_2}")

INFO:__main__:Has 30 records to search the url of


INFO:__main__:	Matched A/S-22/PV.1 to the found: A/S-22/PV.1 and record_id 286240
INFO:__main__:	Matched E/1998/SR.3 to the found: E/1998/SR.3 and record_id 306375
INFO:__main__:	Matched E/1990/SR.16 to the found: E/1990/SR.16-38 and record_id 157261
INFO:__main__:	Matched E/1989/SR.17 to the found: E/1989/SR.17-37 and record_id 149780
INFO:__main__:	Matched E/1988/SR.18 to the found: E/1988/SR.18-42 and record_id 116881
INFO:__main__:	Matched E/1987/SR.20 to the found: E/1987/SR.20-41 and record_id 62300
INFO:__main__:	Matched E/1986/SR.21 to the found: E/1986/SR.21-38and38/Add.1and2 and record_id 156200
INFO:__main__:	Matched A/S-13/PV.1 to the found: A/S-13/PV.1-8 and Annexes and record_id 858165
INFO:__main__:	Matched A/C.5/40/SR.11 to the found: A/C.5/40/SR.11 and record_id 105070
INFO:__main__:	Matched E/1985/SR.28 to the found: E/1985/SR.28-52 and record_id 146003
INFO:__main__:	Matched E/1984/SR.23 to the found: E/1984/SR.23-50 and record_id 141275
INFO:__main__:	Matched E/1984

In [ ]:
print(unmatched_requests_2)

['A/73/PV.102', 'A/S-22/PV.1', 'E/1998/SR.3', 'E/1996/SR.47', 'E/1992/SR.18', 'E/1992/SR.13', 'A/S-13/PV.8', 'A/C.5/40/SR.11', 'E/1985/SR.21', 'E/1985/SR.8', 'E/1981/SR.20', 'E/1980/SR.24', 'E/1979/SR.19', 'E/SR.1859', 'E/SR.1857', 'E/SR.502']


still unmatched are:
- A/73/PV.102 - url does not exist in the MarcXML but in normal search - https://digitallibrary.un.org/record/3824607/files/A_73_PV-102-EN.pdf?ln=en
- A/S-22/PV.1 - no english available
- E/1998/SR.3 - no english available
- E/1996/SR.47 - exists but has no documents
- E/1992/SR.18 - exists but has no documents
- E/1992/SR.13 - exists but has no documents
- A/S-13/PV.8 - exists but has no documents - is part of the downloaded A/S-13/PV.1-8 and Annexes
- A/C.5/40/SR.11 - no english available
- E/1985/SR.21 - exists but has no documents
- E/1985/SR.8 - exists but has no documents
- S/PV.2399andCorr.1 - is called "S/PV.2399 - S/PV.2399ANDCORR.1(OR)"
- E/1981/SR.20 - exists but has no documents
- E/1980/SR.24 - exists but has no documents
- E/1979/SR.19 - url does not exist in the MarcXML but in normal search - https://digitallibrary.un.org/record/4980/files/E_1979_SR-19-EN.pdf?ln=en
- E/SR.1859 - url does not exist in the MarcXML but in normal search - https://digitallibrary.un.org/record/3799195/files/E_SR-1859-EN.pdf?ln=en
- E/SR.1857 - url does not exist in the MarcXML but in normal search - https://digitallibrary.un.org/record/3799195/files/E_SR-1859-EN.pdf?ln=en
- E/SR.502 - url does not exist in the MarcXML but in normal search - https://digitallibrary.un.org/record/3808085/files/E_SR-502-EN.pdf?ln=en

### 3. iteration - create the download links for the missing MarcXML

In [ ]:
url_from_normal_search = [
[
    None,
    "A/73/PV.102",
    "English",
    "https://digitallibrary.un.org/record/3824607/files/A_73_PV-102-EN.pdf?ln=en",
    None,
],[
    None,
    "E/1979/SR.19",
    "English",
    "https://digitallibrary.un.org/record/4980/files/E_1979_SR-19-EN.pdf?ln=en",
    None,
],[
    None,
    "E/SR.1859",
    "English",
    "https://digitallibrary.un.org/record/3799195/files/E_SR-1859-EN.pdf?ln=en",
    None,
],[
    None,
    "E/SR.1857",
    "English",
    "https://digitallibrary.un.org/record/3799194/files/E_SR-1857-EN.pdf?ln=en",
    None,
],[
    None,
    "E/SR.502",
    "English",
    "https://digitallibrary.un.org/record/3808085/files/E_SR-502-EN.pdf?ln=en",
    None,
],
]

In [ ]:
unmatched_requests_2.remove('A/73/PV.102')
unmatched_requests_2.remove('E/1979/SR.19')
unmatched_requests_2.remove('E/SR.1859')
unmatched_requests_2.remove('E/SR.1857')
unmatched_requests_2.remove('E/SR.502')

## Further processing - create a dataframe (and update the metadata)

In [ ]:
data.extend(data_2)
data.extend(url_from_normal_search)

In [ ]:
print(f"we found {len(data)} urls and could not match {len(unmatched_requests_2)}")

we found 1196 urls and could not match 11


save so we have data to debug if needed later

In [ ]:
doc_records = pd.DataFrame(data, columns=['record_id_document', 'document_symbol_found', 'lang_field', 'doc_url', 'date_document'])
doc_records.to_csv(f'{dataframe_dir}{metadata_intermediate_step}.csv', index=False)

reload to have a clean sheet

In [ ]:
speech_records = pd.read_csv(f'{dataframe_dir}{last_metadata_step}.csv')
doc_records = pd.read_csv(f'{dataframe_dir}{metadata_intermediate_step}.csv')

In [ ]:
# this could have been done smarter for example saving it while scraping in an extra field already
doc_records['document_symbol'] = doc_records.document_symbol_found
doc_records.loc[doc_records.document_symbol_found=='E/1990/SR.16-38', 'document_symbol'] = 'E/1990/SR.16'
doc_records.loc[doc_records.document_symbol_found=='E/1989/SR.17-37', 'document_symbol'] = 'E/1989/SR.17'
doc_records.loc[doc_records.document_symbol_found=='E/1988/SR.18-42', 'document_symbol'] = 'E/1988/SR.18'
doc_records.loc[doc_records.document_symbol_found=='E/1987/SR.20-41', 'document_symbol'] = 'E/1987/SR.20'
doc_records.loc[doc_records.document_symbol_found=='E/1986/SR.21-38and38/Add.1and2', 'document_symbol'] = 'E/1986/SR.21'
doc_records.loc[doc_records.document_symbol_found=='A/S-13/PV.1-8 and Annexes', 'document_symbol'] = 'A/S-13/PV.1'
doc_records.loc[doc_records.document_symbol_found=='E/1985/SR.28-52', 'document_symbol'] = 'E/1985/SR.28'
doc_records.loc[doc_records.document_symbol_found=='E/1984/SR.23-50', 'document_symbol'] = 'E/1984/SR.23'
doc_records.loc[doc_records.document_symbol_found=='E/1984/SR.1-22', 'document_symbol'] = 'E/1984/SR.1'
doc_records.loc[doc_records.document_symbol_found=='E/1983/SR.16-42', 'document_symbol'] = 'E/1983/SR.16'
doc_records.loc[doc_records.document_symbol_found=='E/1982/SR.30-58', 'document_symbol'] = 'E/1982/SR.30'
doc_records.loc[doc_records.document_symbol_found=='E/1982/SR.1-29', 'document_symbol'] = 'E/1982/SR.1'
## previously forgotten
doc_records.loc[doc_records.document_symbol_found=='S/PV.2399', 'document_symbol'] = 'S/PV.2399andCorr.1'
doc_records.loc[doc_records.document_symbol_found=='S/PV.2191ANDAdd.1', 'document_symbol'] = 'S/PV.2191andAdd.1'

In [ ]:
metadata = pd.merge(speech_records, doc_records, how='inner', on='document_symbol')
print(f"size: {len(metadata)} (hint at the two duplicates - because its doc_records+2)")
metadata.columns

size: 1198 (hint at the two duplicates - because its doc_records+2)


Index(['record_id', 'speaker', 'speaker_organization', 'document_symbol',
       'date', 'sg_number', 'year', 'month', 'day', 'body', 'sub_body',
       'doc_number', 'doc_type', 'add_part', 'file_prefix', 'file_name_pdf',
       'record_id_document', 'document_symbol_found', 'lang_field', 'doc_url',
       'date_document'],
      dtype='object')

## Clean up duplicates

drop duplicates where all is the same

In [ ]:
dupl_series = metadata.duplicated(subset=["document_symbol", "record_id_document", "doc_url"], keep="first")
display(metadata[dupl_series])
metadata = metadata[~dupl_series]
len(metadata)

,record_id,speaker,speaker_organization,document_symbol,date,sg_number,year,month,day,body,...,doc_number,doc_type,add_part,file_prefix,file_name_pdf,record_id_document,document_symbol_found,lang_field,doc_url,date_document
828,354846,"Boutros-Ghali, Boutros, 1922-2016",UN. Secretary-General,A/47/PV.33,1992-10-12,6,1992,10,12,A,...,47.0,PV.33,NaN,A_1992_47_PV.33_,A_1992_47_PV.33_speeches.pdf,159828.0,A/47/PV.33,English,https://digitallibrary.un.org/record/159828/fi...,1992-10-12
829,351174,"Boutros-Ghali, Boutros, 1922-2016",UN. Secretary-General,A/47/PV.33,1992-10-12,6,1992,10,12,A,...,47.0,PV.33,NaN,A_1992_47_PV.33_,A_1992_47_PV.33_speeches.pdf,159828.0,A/47/PV.33,English,https://digitallibrary.un.org/record/159828/fi...,1992-10-12
830,351174,"Boutros-Ghali, Boutros, 1922-2016",UN. Secretary-General,A/47/PV.33,1992-10-12,6,1992,10,12,A,...,47.0,PV.33,NaN,A_1992_47_PV.33_,A_1992_47_PV.33_speeches.pdf,159828.0,A/47/PV.33,English,https://digitallibrary.un.org/record/159828/fi...,1992-10-12


1195

list all further duplicates

In [ ]:
dupl_series = metadata.duplicated(subset=["document_symbol"], keep=False)
display(metadata[dupl_series])

,record_id,speaker,speaker_organization,document_symbol,date,sg_number,year,month,day,body,...,doc_number,doc_type,add_part,file_prefix,file_name_pdf,record_id_document,document_symbol_found,lang_field,doc_url,date_document
885,326292,"Pérez de Cuéllar, Javier, 1920-2020",UN. Secretary-General,A/S-15/PV.1,1988-05-31,5,1988,5,31,A,...,NaN,PV.1,NaN,A_1988_S-15_PV.1_,A_1988_S-15_PV.1_speeches.pdf,761784.0,A/S-15/PV.1,English,https://digitallibrary.un.org/record/761784/fi...,1989
886,326292,"Pérez de Cuéllar, Javier, 1920-2020",UN. Secretary-General,A/S-15/PV.1,1988-05-31,5,1988,5,31,A,...,NaN,PV.1,NaN,A_1988_S-15_PV.1_,A_1988_S-15_PV.1_speeches.pdf,39664.0,A/S-15/PV.1,English,https://digitallibrary.un.org/record/39664/fil...,1988-05-31
909,312127,"Pérez de Cuéllar, Javier, 1920-2020",UN. Secretary-General,A/S-14/PV.1,1986-09-17,5,1986,9,17,A,...,NaN,PV.1,NaN,A_1986_S-14_PV.1_,A_1986_S-14_PV.1_speeches.pdf,857792.0,A/S-14/PV.1,English,https://digitallibrary.un.org/record/857792/fi...,1986
910,312127,"Pérez de Cuéllar, Javier, 1920-2020",UN. Secretary-General,A/S-14/PV.1,1986-09-17,5,1986,9,17,A,...,NaN,PV.1,NaN,A_1986_S-14_PV.1_,A_1986_S-14_PV.1_speeches.pdf,120798.0,A/S-14/PV.1,English,https://digitallibrary.un.org/record/120798/fi...,1986-09-17


There is a document duplication for A/S-15/PV.1 on UN site

In [ ]:
# keep only one A/S-15/PV.1 -  there is a document duplication on UN site
# display(metadata[metadata.document_symbol.isin(['A/S-15/PV.1'])])
dupl_index = metadata[metadata.record_id_document == 761784].index
metadata = metadata.drop(index=dupl_index)
len(metadata)

1194

more duplicates

In [ ]:
# keep only one A/S-15/PV.1 -  there is a document duplication on UN site
# display(metadata[metadata.document_symbol.isin(['A/S-14/PV.1'])])
dupl_index = metadata[metadata.record_id_document == 857792].index
metadata = metadata.drop(index=dupl_index)
len(metadata)

1193

drop one further document that says its english but is actually french

In [ ]:
dupl_index = metadata[metadata.document_symbol == 'E/2011/SR.20(A)'].index
metadata = metadata.drop(index=dupl_index)
len(metadata)

1192

# Save data

In [ ]:
metadata.reset_index(drop=True, inplace=True)
metadata.to_csv(f'{dataframe_dir}{current_metadata_step}.csv', index=False)